In [1]:
import re
import qiskit
import json
import time

import numpy as np
from typing import List
from qiskit import *
import time

from fast_generator import fc_tree_commute_recur_lookahead_fast

from benchmarks.UCCSD_entanglers import generate_UCCSD_entanglers
from circuit_generator import construct_qcc_circuit

import numpy as np
from rustiq import pauli_network_synthesis, Metric
from rustiq.utils import entangling_count, entangling_depth

In [2]:
#Compare json files in a specific folder
import os, json
def run_experiment_folder(folder_path = None, filename = None, save_output = False, threshold = 1):

    results = []
    if filename == None:
        file_list = os.listdir(folder_path)
    else:
        file_list = [filename]
    # Iterate over all files in the folder
    for filename in file_list:
        # Check if the file is a JSON file
        if filename.endswith(".json"):
            # Print the filename
            print(filename)
            with open(folder_path + '/' + filename, "r") as file:
                paulis = json.load(file)

            # Function to check if a string contains only 'I' characters
            def is_all_identity(pauli):
                return all(char == 'I' for char in pauli)

            # Filter the list to remove all identity Paulis
            test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)]
            test_params = [0.01 * i for i in range(len(test_paulis))]

            # Measure time for our method
            start_time = time.time()
            opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_recur_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, threshold=threshold)
            end_time = time.time()
            our_time = end_time - start_time

            start_time = time.time()
            opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_recur_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, threshold=threshold)
            opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
            end_time = time.time()
            combined_time = end_time - start_time
            
            # Measure time for Qiskit method
            start_time = time.time()
            origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
            origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
            end_time = time.time()
            qiskit_time = end_time - start_time

            # Measure time for RustiQ method
            start_time = time.time()
            circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True, fix_clifford=True)
            end_time = time.time()
            rustiq_time = end_time - start_time
        
            # Collect results
            result = {
                "num_paulis": len(test_paulis),
                "times": {
                    "our_time": our_time,
                    "combined_time": combined_time,
                    "qiskit_time": qiskit_time,
                    "rustiq_time": rustiq_time
                },
                "gate_counts": {
                    "our_method": opt_qc_f.count_ops().get('cx', 0),
                    "combined_method": opt_qiskit.count_ops().get('cx', 0),
                    "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
                    "rustiq_method": entangling_count(circuit)
                },
                "test_paulis_file": f'benchmarks/results/test_' + filename
            }
            print(result)
            results.append(result)
            if save_output == True:
                # Save test_paulis to a separate JSON file
                with open(f'benchmarks/results/test_' + filename, 'w') as paulis_file:
                    json.dump(test_paulis, paulis_file, indent=4)
    

    

In [3]:
#Compare a given list of paulis
def run_experiment_paulis(test_paulis, test_params = None, save_output = False, threshold = 1, filename = "Paulis"):

    results = []

    def is_all_identity(pauli):
        return all(char == 'I' for char in pauli)
    paulis = test_paulis
    # Filter the list to remove all identity Paulis
    test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)]
    if test_params is None:
        test_params = [0.01 * i for i in range(len(test_paulis))]

    # Measure time for our method
    start_time = time.time()
    opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_recur_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, threshold=threshold)
    end_time = time.time()
    our_time = end_time - start_time

    start_time = time.time()
    opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_recur_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, threshold=threshold)
    opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
    end_time = time.time()
    combined_time = end_time - start_time
    
    # Measure time for Qiskit method
    start_time = time.time()
    origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
    origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
    end_time = time.time()
    qiskit_time = end_time - start_time

    # Measure time for RustiQ method
    start_time = time.time()
    circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True, fix_clifford=True)
    end_time = time.time()
    rustiq_time = end_time - start_time
    
    # Collect results
    result = {
        "num_paulis": len(test_paulis),
        "times": {
            "our_time": our_time,
            "combined_time": combined_time,
            "qiskit_time": qiskit_time,
            "rustiq_time": rustiq_time
        },
        "gate_counts": {
            "our_method": opt_qc_f.count_ops().get('cx', 0),
            "combined_method": opt_qiskit.count_ops().get('cx', 0),
            "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
            "rustiq_method": entangling_count(circuit)
        },
        "test_paulis_file": f'benchmarks/results/test_' + filename
    }
    print(result)
    results.append(result)
    if save_output == True:
        # Save test_paulis to a separate JSON file
        with open(f'benchmarks/results/test_' + filename, 'w') as paulis_file:
            json.dump(test_paulis, paulis_file, indent=4)
    return sorted_entanglers_f
    

    

In [5]:
#first compare the UCCSD ansatz
electrons_list = [2, 2, 4, 6, 8, 10, 12]
orbitals_list = [4, 6, 8, 12, 16, 20, 24]
#First evaluate the UCCSD ansatz:
for i, (x, y) in enumerate(zip(electrons_list, orbitals_list)):
    test_paulis = generate_UCCSD_entanglers(x, y)
    entanglers = run_experiment_paulis(test_paulis = test_paulis, save_output = False, threshold = 1)

{'num_paulis': 24, 'times': {'our_time': 0.15991997718811035, 'combined_time': 0.6117300987243652, 'qiskit_time': 0.18233180046081543, 'rustiq_time': 0.002371072769165039}, 'gate_counts': {'our_method': 27, 'combined_method': 23, 'qiskit_method': 41, 'rustiq_method': 33}, 'test_paulis_file': 'benchmarks/results/test_Paulis'}
{'num_paulis': 80, 'times': {'our_time': 0.4239389896392822, 'combined_time': 0.6682000160217285, 'qiskit_time': 0.4792649745941162, 'rustiq_time': 0.008862972259521484}, 'gate_counts': {'our_method': 119, 'combined_method': 106, 'qiskit_method': 181, 'rustiq_method': 162}, 'test_paulis_file': 'benchmarks/results/test_Paulis'}
{'num_paulis': 320, 'times': {'our_time': 2.189055919647217, 'combined_time': 2.999169111251831, 'qiskit_time': 2.184493064880371, 'rustiq_time': 0.11479330062866211}, 'gate_counts': {'our_method': 507, 'combined_method': 448, 'qiskit_method': 1003, 'rustiq_method': 794}, 'test_paulis_file': 'benchmarks/results/test_Paulis'}


KeyboardInterrupt: 

In [7]:
#Then compare the Hamiltonian simulation paulis in HS_paulis folder
run_experiment_folder(folder_path = "benchmarks/HS_paulis", save_output = False, threshold = 1)

LiH.json
{'num_paulis': 61, 'times': {'our_time': 0.41883397102355957, 'combined_time': 0.5098741054534912, 'qiskit_time': 0.2713038921356201, 'rustiq_time': 0.006908893585205078}, 'gate_counts': {'our_method': 76, 'combined_method': 74, 'qiskit_method': 180, 'rustiq_method': 114}, 'test_paulis_file': 'benchmarks/results/test_LiH.json'}
H2O.json
{'num_paulis': 184, 'times': {'our_time': 1.826855182647705, 'combined_time': 2.0217039585113525, 'qiskit_time': 1.5432188510894775, 'rustiq_time': 0.05791306495666504}, 'gate_counts': {'our_method': 277, 'combined_method': 274, 'qiskit_method': 786, 'rustiq_method': 352}, 'test_paulis_file': 'benchmarks/results/test_H2O.json'}
benzene.json
{'num_paulis': 1254, 'times': {'our_time': 18.847551107406616, 'combined_time': 22.20553994178772, 'qiskit_time': 12.602912902832031, 'rustiq_time': 7.250926733016968}, 'gate_counts': {'our_method': 2479, 'combined_method': 2470, 'qiskit_method': 7602, 'rustiq_method': 3356}, 'test_paulis_file': 'benchmarks/

In [11]:
#Then compare the MAXCUT paulis in max_cut_paulis folder

run_experiment_folder(folder_path = "benchmarks/max_cut_paulis", save_output = False, threshold = 1)

max_cut_benchmark_n14_e60_l5.json
{'num_paulis': 370, 'times': {'our_time': 17.008366107940674, 'combined_time': 14.982296228408813, 'qiskit_time': 0.5040137767791748, 'rustiq_time': 0.09271717071533203}, 'gate_counts': {'our_method': 355, 'combined_method': 355, 'qiskit_method': 588, 'rustiq_method': 463}, 'test_paulis_file': 'benchmarks/results/test_max_cut_benchmark_n14_e60_l5.json'}
max_cut_benchmark_n10_e45_l20.json
{'num_paulis': 1100, 'times': {'our_time': 28.745305061340332, 'combined_time': 30.85427188873291, 'qiskit_time': 2.1607210636138916, 'rustiq_time': 0.5809931755065918}, 'gate_counts': {'our_method': 1001, 'combined_method': 1001, 'qiskit_method': 1798, 'rustiq_method': 1119}, 'test_paulis_file': 'benchmarks/results/test_max_cut_benchmark_n10_e45_l20.json'}


In [12]:
#since in the maxcut problems there are many commuting terms and it's slow to search through all of them,
#we can set the threshold to 2 to early stop the search once found a simplified pauli of weight 2
#you can see the run time difference, for the second benchmark runtime reduces from 30second to 7 seconds.

run_experiment_folder(folder_path = "benchmarks/max_cut_paulis", save_output = False, threshold = 2)

max_cut_benchmark_n14_e60_l5.json
{'num_paulis': 370, 'times': {'our_time': 8.170368909835815, 'combined_time': 8.644093990325928, 'qiskit_time': 0.4975261688232422, 'rustiq_time': 0.09200096130371094}, 'gate_counts': {'our_method': 380, 'combined_method': 380, 'qiskit_method': 588, 'rustiq_method': 462}, 'test_paulis_file': 'benchmarks/results/test_max_cut_benchmark_n14_e60_l5.json'}
max_cut_benchmark_n10_e45_l20.json
{'num_paulis': 1100, 'times': {'our_time': 5.964715003967285, 'combined_time': 7.297049045562744, 'qiskit_time': 1.737131118774414, 'rustiq_time': 0.5721719264984131}, 'gate_counts': {'our_method': 1071, 'combined_method': 1071, 'qiskit_method': 1798, 'rustiq_method': 1119}, 'test_paulis_file': 'benchmarks/results/test_max_cut_benchmark_n10_e45_l20.json'}
